In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
DATA_DIR = r'C:\Users\User\Downloads\Real-life_Deception_Detection_2016\Frames\Openface_features'

In [3]:
def create_openface_dataset(data_dir):
    x = []
    y = []
    filenames = []
    
    for filename in tqdm(os.listdir(data_dir)):
        fn=os.path.splitext(filename)[0] # goes through files names without extension
        if 'of_details' not in fn:
            # filenames.append(fn)
            openface_df = pd.read_csv(os.path.join(data_dir,filename))
            # fill zeroes with mean values where openface failed to detect faces 
            openface_df.loc[openface_df[' success'] == 0] = openface_df.loc[openface_df[' success'] == 0].replace(0, openface_df.loc[openface_df[' success'] == 1].mean())
            # remove some irrelevant columns
            openface_df = openface_df.loc[:, ~openface_df.columns.isin(['frame', ' face_id', ' timestamp', ' confidence', ' success'])]

            total_features=None
            mean_features = (np.mean(openface_df, axis=0))
            std_features = (np.std(openface_df, axis=0))
            max_features = (np.max(openface_df, axis=0))
            min_features = (np.min(openface_df, axis=0))

            # join several features together
            feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                    
            #feature = np.concatenate((mean_features, std_features, min_features), axis=None)
            #feature = np.concatenate((mean_features, min_features, max_features), axis=None)
            #feature = np.concatenate((max_features, std_features), axis=None)
            #feature=max_features

            total_features=feature
            
            is_nan = False
            for j in range(len(total_features)):
                if np.isnan(total_features[j]):
                    is_nan = True
            
            if is_nan == False and total_features is not None:
                x.append(total_features)
                if filename[6:11] == 'truth':
                    y.append(1)
                else:
                    y.append(0)
                # print(filename + '_' + i)
                filenames.append(fn)

            if is_nan == True:
                print(filename)

    x=np.array(x)
    y=np.array(y)
    
    print(x.shape,y.shape)
    return x,y,filenames

X, y, filenames = create_openface_dataset(os.path.join(DATA_DIR))
# x_test, y_test = create_openface_dataset(os.path.join(DATA_DIR, 'Val_AFEW'))

 78%|███████▊  | 185/238 [00:25<00:07,  6.75it/s]

trial_truth_033.csv


 79%|███████▊  | 187/238 [00:26<00:08,  6.25it/s]

trial_truth_034.csv


 81%|████████  | 193/238 [00:27<00:06,  6.84it/s]

trial_truth_037.csv


100%|██████████| 238/238 [00:31<00:00,  7.46it/s]

(114, 1316) (114,)


trial_truth_033, trial_truth_034, trial_truth_037 - files with NaN values

### Openface Train/Test Split

### Classification

In [5]:
import xgboost as xgb
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

### Cross_val_score

In [6]:
xgb_clf = xgb.XGBClassifier(use_label_encoder=False)
scores = cross_val_score(xgb_clf, X, y, cv=StratifiedKFold(10))
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

[1.         1.         1.         0.91666667 1.         1.
 0.90909091 1.         0.90909091 1.        ]
0.97 accuracy with a standard deviation of 0.04


In [7]:
len(X)

114

In [8]:
len(y)

114

In [38]:
rfc_clf = RandomForestClassifier(n_estimators = 1000)
scores = cross_val_score(rfc_clf, X, y, cv=10)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

[1.         1.         0.91304348 1.         1.         1.
 0.95652174 1.         0.90909091 1.        ]
0.98 accuracy with a standard deviation of 0.04
